In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../..'))
	print(os.getcwd())
except:
	pass


 # 1. 事前に準備済みのK8s環境を確認

 ## K8s

In [1]:
%%bash
az aks show -g rg_bdc -n o9o9aks


Name     Location    ResourceGroup    KubernetesVersion    ProvisioningState    Fqdn
-------  ----------  ---------------  -------------------  -------------------  -----------------------------------------------------
o9o9aks  japaneast   rg_bdc           1.14.6               Succeeded            o9o9aks-rgbdc-745830-7ebfd297.hcp.japaneast.azmk8s.io


In [2]:
%%bash
az aks browse -g rg_bdc --listen-address 0.0.0.0 -n o9o9aks


Process is interrupted.


 ## Node

In [3]:
%%bash
kubectl get node -o wide


NAME                       STATUS   ROLES   AGE   VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION      CONTAINER-RUNTIME
aks-nodepool1-15232104-0   Ready    agent   63d   v1.14.6   10.240.0.4    <none>        Ubuntu 16.04.6 LTS   4.15.0-1059-azure   docker://3.0.6
aks-nodepool1-15232104-1   Ready    agent   63d   v1.14.6   10.240.0.6    <none>        Ubuntu 16.04.6 LTS   4.15.0-1059-azure   docker://3.0.6
aks-nodepool1-15232104-2   Ready    agent   63d   v1.14.6   10.240.0.5    <none>        Ubuntu 16.04.6 LTS   4.15.0-1059-azure   docker://3.0.6


In [4]:
%%bash
az vm list -g MC_RG_BDC_O9O9AKS_JAPANEAST -d


Name                      ResourceGroup                PowerState    PublicIps    Fqdns    Location    Zones
------------------------  ---------------------------  ------------  -----------  -------  ----------  -------
aks-nodepool1-15232104-0  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast
aks-nodepool1-15232104-1  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast
aks-nodepool1-15232104-2  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast


In [5]:
%%bash
kubectl describe node


Name:               aks-nodepool1-15232104-0
Roles:              agent
Labels:             agentpool=nodepool1
                    beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/instance-type=Standard_B16ms
                    beta.kubernetes.io/os=linux
                    failure-domain.beta.kubernetes.io/region=japaneast
                    failure-domain.beta.kubernetes.io/zone=1
                    kubernetes.azure.com/cluster=MC_rg_bdc_o9o9aks_japaneast
                    kubernetes.azure.com/role=agent
                    kubernetes.io/arch=amd64
                    kubernetes.io/hostname=aks-nodepool1-15232104-0
                    kubernetes.io/os=linux
                    kubernetes.io/role=agent
                    node-role.kubernetes.io/agent=
                    storageprofile=managed
                    storagetier=Premium_LRS
Annotations:        node.alpha.kubernetes.io/ttl: 0
                    volumes.kubernetes.io/controller-managed-attach-det

 # 2. operator作成

In [6]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/AG/1.operator.yaml


# apiVersion: v1
# kind: Namespace
# metadata: {name: mssql-ag}
---
apiVersion: v1
kind: ServiceAccount
metadata: {name: mssql-operator, namespace: mssql-ag}
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata: {name: mssql-operator-ag}
rules:
- apiGroups: ['']
  resources: [serviceaccounts, services]
  verbs: [create, get, update, delete]
- apiGroups: [batch]
  resources: [jobs]
  verbs: [create, get, update, delete]
- apiGroups: [rbac.authorization.k8s.io]
  resources: [roles, rolebindings]
  verbs: [create, get, update, delete]
- apiGroups: [apps]
  resources: [statefulsets]
  verbs: [create, delete, get, update]
- apiGroups: ['']
  resources: [configmaps, endpoints, secrets]
  verbs: [create, get, update, watch, delete]
- apiGroups: ['']
  resources: [pods]
  verbs: [get, list, update]
- apiGroups: [apiextensions.k8s.io]
  resources: [customresourcedefinitions]
  verbs: [create]
- apiGroups: [apiextensions.k8s.io]
  resourceNames: [sqlservers.mssql.microsoft.com

 # 3. pod作成

In [7]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/AG/2.sqlserver.yaml


apiVersion: mssql.microsoft.com/v1
kind: SqlServer
metadata:
  labels: {name: mssql1, type: sqlservr}
  name: mssql1
  namespace: mssql-ag
spec:
  acceptEula: true
  agentsContainerImage: mcr.microsoft.com/mssql/ha:2019-CTP3.2-ubuntu
  availabilityGroups: [mssql-ag]
  instanceRootVolumeClaimTemplate:
    accessModes: [ReadWriteOnce]
    resources:
      requests: {storage: 5Gi}
    storageClass: default
  saPassword:
    secretKeyRef: {key: sapassword, name: sql-secrets}
  sqlServerContainer: {image: 'mcr.microsoft.com/mssql/server:2019-CTP3.2-ubuntu'}
---
apiVersion: v1
kind: Service
metadata: {name: mssql1, namespace: mssql-ag}
spec:
  ports:
  - {name: tds, port: 1433}
  selector: {name: mssql1, type: sqlservr}
  type: LoadBalancer
---
apiVersion: mssql.microsoft.com/v1
kind: SqlServer
metadata:
  labels: {name: mssql2, type: sqlservr}
  name: mssql2
  namespace: mssql-ag
spec:
  acceptEula: true
  agentsContainerImage: mcr.microsoft.com/mssql/ha:2019-CTP3.2-ubuntu
  availabilityGro

 # 4. Service定義

In [8]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/AG/3.ag-services.yaml


apiVersion: v1
kind: Service
metadata: {annotations: null, name: ag-primary, namespace: mssql-ag}
spec:
  ports:
  - {name: tds, port: 1433, targetPort: 1433}
  selector: {role.ag.mssql.microsoft.com/mssql-ag: primary, type: sqlservr}
  type: LoadBalancer
---
apiVersion: v1
kind: Service
metadata: {annotations: null, name: ag-secondary, namespace: mssql-ag}
spec:
  ports:
  - {name: tds, port: 1433}
  selector: {role.ag.mssql.microsoft.com/mssql-ag: secondary,
    type: sqlservr}
  type: LoadBalancer
---

 # 5. 環境確認

In [9]:
%%bash
kubectl get pod -o wide -n mssql-ag


NAME                              READY   STATUS      RESTARTS   AGE   IP            NODE                       NOMINATED NODE   READINESS GATES
mssql-initialize-mssql1-bt9vq     0/1     Completed   0          10h   10.244.0.48   aks-nodepool1-15232104-1   <none>           <none>
mssql-initialize-mssql2-5gmnl     0/1     Completed   0          10h   10.244.1.76   aks-nodepool1-15232104-2   <none>           <none>
mssql-initialize-mssql3-r7568     0/1     Completed   0          10h   10.244.2.65   aks-nodepool1-15232104-0   <none>           <none>
mssql-operator-65459979cf-djbpj   1/1     Running     0          10h   10.244.2.64   aks-nodepool1-15232104-0   <none>           <none>
mssql1-0                          2/2     Running     0          10h   10.244.2.66   aks-nodepool1-15232104-0   <none>           <none>
mssql2-0                          2/2     Running     0          10h   10.244.0.49   aks-nodepool1-15232104-1   <none>           <none>
mssql3-0                          2/2  

In [10]:
%%bash
kubectl get pvc -o wide -n mssql-ag
# kubectl describe pvc -n mssql-ag


NAME                     STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
instance-root-mssql1-0   Bound    pvc-9c4ab426-0d30-11ea-8d22-169104339902   5Gi        RWO            default        10h
instance-root-mssql2-0   Bound    pvc-9de35eda-0d30-11ea-8d22-169104339902   5Gi        RWO            default        10h
instance-root-mssql3-0   Bound    pvc-9faea515-0d30-11ea-8d22-169104339902   5Gi        RWO            default        10h


In [11]:
%%bash
az disk list -g MC_RG_BDC_O9O9AKS_JAPANEAST


Name                                                                ResourceGroup                Location    Zones    Sku           OsType    SizeGb    ProvisioningState
------------------------------------------------------------------  ---------------------------  ----------  -------  ------------  --------  --------  -------------------
aks-nodepool1-15232104-0_OsDisk_1_233442b330ce48aeb803829051658ad3  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
aks-nodepool1-15232104-1_OsDisk_1_1f8b8034fcce4ee2b97c024821b6d8bf  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
aks-nodepool1-15232104-2_OsDisk_1_e0e8ca95bf704caeade77ebf95d47a82  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
kubernetes-dynamic-pvc-246beaa7-0d88-11ea-8d22-169104339902         MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Standard_LRS            10        Succeeded
kubernetes

In [12]:
%%bash
kubectl get service -o wide -n mssql-ag

# AG確認（DMV）


NAME           TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)             AGE   SELECTOR
ag-primary     LoadBalancer   10.0.28.69     40.115.179.101   1433:30697/TCP      10h   role.ag.mssql.microsoft.com/mssql-ag=primary,type=sqlservr
ag-secondary   LoadBalancer   10.0.129.38    20.43.76.137     1433:30559/TCP      10h   role.ag.mssql.microsoft.com/mssql-ag=secondary,type=sqlservr
mssql-ag       ClusterIP      None           <none>           1433/TCP,5022/TCP   10h   ag-service.mssql.microsoft.com/mssql-ag=
mssql1         LoadBalancer   10.0.172.154   40.115.182.249   1433:32465/TCP      10h   name=mssql1,type=sqlservr
mssql2         LoadBalancer   10.0.101.121   40.115.154.147   1433:30786/TCP      10h   name=mssql2,type=sqlservr
mssql3         LoadBalancer   10.0.37.93     40.115.178.136   1433:31683/TCP      10h   name=mssql3,type=sqlservr


 # 6. AG確認（DMV）

In [16]:
%%bash
sqlcmd -S 40.115.179.101 -U sa -P $PASSWORD -Q 'select r.replica_server_name, rs.is_primary_replica, rs.synchronization_health_desc FROM sys.dm_hadr_database_replica_states rs JOIN sys.availability_replicas r ON r.group_id = rs.group_id AND r.replica_id = rs.replica_id'


replica_server_name                                                                                                                                                                                                                                              is_primary_replica synchronization_health_desc                                 
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ------------------ ------------------------------------------------------------
mssql3-0                                                                                                                                                                                                                                                                          0 NOT_HEALTHY                                       

In [17]:
%%bash
sqlcmd -S 40.115.179.101 -U sa -P $PASSWORD -Q 'select @@servername'



--------------------------------------------------------------------------------------------------------------------------------
mssql1-0                                                                                                                        

(1 rows affected)


In [18]:
%%bash
sqlcmd -S 20.43.76.137 -U sa -P $PASSWORD -Q 'select @@servername'



--------------------------------------------------------------------------------------------------------------------------------
mssql3-0                                                                                                                        

(1 rows affected)


 # 6. AG確認（SSMS）

 # 6. AG確認（K8s Dashboard）

In [19]:
%%bash
az aks browse -g rg_bdc --listen-address 0.0.0.0 -n o9o9aks


Process is interrupted.


 # 7.セカンダリから接続（ReadOnlyでないとエラーになる）

In [20]:
%%bash
sqlcmd -S 20.43.76.137 -U sa -P $PASSWORD -Q 'use agdb; select * from tab01'


Msg 978, Level 14, State 1, Server mssql2-0, Line 1
The target database ('agdb') is in an availability group and is currently accessible for connections when the application intent is set to read only. For more information about application intent, see SQL Server Books Online.


In [21]:
%%bash
sqlcmd -S 20.43.76.137 -U sa -P $PASSWORD -Q 'use agdb; select * from tab01' -K READONLY


Changed database context to 'agdb'.
id          name                                                                                                                                                                                                                                                            
----------- ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
          1 AG on K8s!                                                                                                                                                                                                                                                      

(1 rows affected)


 # 8. AGの機能でフェールオーバー（エラーになる）

In [22]:
%%bash
sqlcmd -S 40.115.179.101 -U sa -P $PASSWORD -Q 'ALTER AVAILABILITY GROUP agdb FAILOVER'


Msg 15151, Level 16, State 1, Server mssql1-0, Line 1
Cannot alter the availability group 'agdb', because it does not exist or you do not have permission.


 # 9. K8sのJobでフェールオーバー

In [23]:
%%bash
# kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/AG/4-1.failover_to_mssql1-0.yaml -n mssql-ag
kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/AG/4-2.failover_to_mssql2-0.yaml -n mssql-ag
# kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/AG/4-3.failover_to_mssql3-0.yaml -n mssql-ag


serviceaccount/manual-failover created
role.rbac.authorization.k8s.io/manual-failover created
rolebinding.rbac.authorization.k8s.io/manual-failover created
job.batch/manual-failover created


In [24]:
%%bash
kubectl get jobs -n mssql-ag


NAME                      COMPLETIONS   DURATION   AGE
manual-failover           1/1           5s         65s
mssql-initialize-mssql1   1/1           12m        11h
mssql-initialize-mssql2   1/1           12m        11h
mssql-initialize-mssql3   1/1           12m        11h


 # 10. AG確認

In [25]:
%%bash
sqlcmd -S 40.115.179.101 -U sa -P $PASSWORD -Q 'select r.replica_server_name, rs.is_primary_replica, rs.synchronization_health_desc FROM sys.dm_hadr_database_replica_states rs JOIN sys.availability_replicas r ON r.group_id = rs.group_id AND r.replica_id = rs.replica_id'


replica_server_name                                                                                                                                                                                                                                              is_primary_replica synchronization_health_desc                                 
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ------------------ ------------------------------------------------------------
mssql3-0                                                                                                                                                                                                                                                                          0 HEALTHY                                           

In [26]:
%%bash
sqlcmd -S 40.115.179.101 -U sa -P $PASSWORD -Q 'select @@servername'



--------------------------------------------------------------------------------------------------------------------------------
mssql2-0                                                                                                                        

(1 rows affected)


In [27]:
%%bash
sqlcmd -S 20.43.76.137 -U sa -P $PASSWORD -Q 'select @@servername'



--------------------------------------------------------------------------------------------------------------------------------
mssql3-0                                                                                                                        

(1 rows affected)
